In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import ee
import geemap
import geetools
from datetime import datetime

In [2]:
ee.Initialize(project='ee-jonstar')

In [3]:
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

## CONUS Geometry

In [4]:
conus = ee.ImageCollection("GRIDMET/DROUGHT").filterDate('2000-01-01', '2001-01-01')
geometry = conus.geometry();

In [5]:
u_lon = -76.6122
u_lat = 39.2904
u_poi = ee.Geometry.Point(u_lon, u_lat)
roi = u_poi.buffer(2e2)

## Landsat Initialization

In [6]:
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
landsat_US = landsat.filterDate('2023-01-01', '2024-08-17').filterBounds(roi)

In [7]:
landsat_US.first().projection()

## GOES Initialization

In [8]:
GOES = ee.ImageCollection("NOAA/GOES/16/MCMIPC").filterDate('2023-01-01', '2024-08-17')

## Filtering images by time

In [9]:
join = ee.Join.inner()
filterNear = ee.Filter.maxDifference(difference=60000, leftField='system:time_start', rightField='system:time_start')

In [10]:
col_near = ee.ImageCollection(join.apply(landsat_US, GOES, filterNear))

In [11]:
col_near.size().getInfo()

27

In [12]:
landsat_US.size().getInfo()

27

## Helper Functions

In [13]:
# Applies scale and offset factors for Landsat band 10 LST
def scale_and_offset_LST(image):
    return image.select('ST_B10').multiply(0.00341802).add(149)

# Applies scale and offset factors for Landsat reflectance bands
def scale_and_offset_Landsat(image):
    return image.select(['SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']).multiply(2.75e-05).add(-0.2)

In [14]:
# Applies scale and offset factors for GOES imagery
# All scale and offset values were taken straight from GEE to save compute
def scale_and_offset_GOES(image, band):
    reflectances = image.select(['CMI_C01', 'CMI_C02', 'CMI_C03', 'CMI_C05', 'CMI_C06']).multiply(0.0002442)
    brightness_temps = image.select(['CMI_C14', 'CMI_C15']).multiply(0.039316241).add(173.14999)
    
    return reflectances.addBands(srcImg=brightness_temps, names=['CMI_C14', 'CMI_C15'])


def scale_and_offset_GOES2(image, band):
    if band == 14:
        scale = 0.059850749
        offset = 96.190002
        #print(scale, offset)
        return image.select('CMI_C14').multiply(scale).add(offset)
    elif band == 15:
        #scale = image.get('CMI_C15_scale').getInfo()
        scale = 0.05956082
        #offset = image.get('CMI_C15_offset').getInfo()
        offset = 97.379997
        #print(scale, offset)
        return image.select('CMI_C15').multiply(scale).add(offset)
    else:
        return image

In [15]:
# Preprocessing function
def process_feature(feature):
    #####################################
    # Landsat portion
    
    # Scaling and offset
    image1 = ee.Image(feature.get('primary'))
    landsat_LST = scale_and_offset_LST(image1)
    landsat_image = scale_and_offset_Landsat(image1)
    # Add cloud mask back in
    landsat_image = landsat_image.addBands(srcImg=landsat_LST, names=['ST_B10'])\
                                .addBands(srcImg=image1, names=['QA_PIXEL'])

    ######################################
    # GOES portion

    # Scaling and offset
    image2 = ee.Image(feature.get('secondary'))
    GOES_image = scale_and_offset_GOES(image2, 14)

    ########################################
    # Mapping for both
    
    def get_median_time(f):
        #return f.set('timestamp', image1.get('system:time_start'))
        #ee.Image(f.copyProperties(image1, ['system:time_start']))
        first_time = ee.Number(image1.get('system:time_start'))
        second_time = ee.Number(image2.get('system:time_start'))
        median_time = ee.Number(first_time.add(second_time).divide(2))
        
        return f.set('timestamp', median_time)
        
    def map_date_GOES(f):
        #return f.set('timestamp', image2.get('system:time_start'))
        return ee.Image(f.copyProperties(image2, ['system:time_start']))

    ##########################################
    # Add timestamp back in
    landsat_image = get_median_time(landsat_image)
    #GOES_image = map_date_GOES(GOES_image)

    ##########################################
    
    image = landsat_image.addBands(srcImg=GOES_image, names=['CMI_C01', 'CMI_C02', 'CMI_C03', 'CMI_C05',
                                                             'CMI_C06', 'CMI_C14', 'CMI_C15']).cast({'QA_PIXEL':'double'})
    return image

## Produce geotiffs and their respective times

In [16]:
processed = col_near.map(process_feature)

In [17]:
times = processed.aggregate_array('timestamp')

In [18]:
def times_to_features(num):
    return ee.Feature(None, {'value': num})

features = ee.FeatureCollection(times.map(times_to_features))

In [19]:
task = ee.batch.Export.table.toDrive(
    collection=features,
    description='times',
    folder='GOES_Landsat_CONUS',
    fileFormat='CSV',
)
task.start()

In [24]:
first = ee.Image(processed.toList(3).get(0))

In [25]:
first

In [20]:
num = processed.size().getInfo()
img_list = processed.toList(num)

for i in list(range(num)):
    img = ee.Image(img_list.get(i))
    
    task = ee.batch.Export.image.toDrive(
        img, description=f'GOES_Landsat_image_{i}', fileFormat='GeoTIFF', folder='GOES_Landsat_CONUS',
                dimensions='7677x7908', crs='EPSG:32618', crsTransform=[30, 0, 220185, 0, -30, 4426815])
    task.start()

## Try from gee code

In [23]:
landsat_image = landsat_US.select('ST_B10').mosaic().multiply(0.00341802).add(149)
GOES_image14 = scale_and_offset_GOES(GOES.first().clip(landsat_US.first().geometry()), 14)
GOES_image15 = scale_and_offset_GOES(GOES.first().clip(landsat_US.first().geometry()), 15)

In [89]:
Map = geemap.Map(center=[u_lat, u_lon], zoom=4)
Map.add_basemap("HYBRID")

#Map.addLayer(first.select('ST_B10'), {'min':275, 'max':350,
 #   'palette':['#fef0d9', '#fdcc8a', '#fc8d59', '#e34a33', '#b30000']}, 'LST')
#Map.addLayer(first.select('CMI_C14'), {'min':96, 'max':342,
 #   'palette':['#f7f7f7', '#cccccc', '#969696', '#636363', '#252525']}, 'GOES14')
#Map.addLayer(img_try, {'min':96, 'max':342,
#    'palette':['#f7f7f7', '#cccccc', '#969696', '#636363', '#252525']}, 'GOES14_try')
#Map.addLayer(first_feature.select('ST_B10'), {'min':275, 'max':350,
#'palette':['#fef0d9', '#fdcc8a', '#fc8d59', '#e34a33', '#b30000']}, 'Landsat')
#Map.addLayer(GOES_image14, {'min':96, 'max':342,
#                         'palette':['#f7f7f7', '#cccccc', '#969696', '#636363', '#252525']}, 'GOES14')
#Map.addLayer(GOES_image15, {'min':97, 'max':342,
 #                        'palette':['#f7f7f7', '#cccccc', '#969696', '#636363', '#252525']}, 'GOES15')

In [90]:
Map

Map(center=[39.2904, -76.6122], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…